In [208]:
import bs4
import lxml
import requests
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt

In [209]:
# Use requests library query power outages.
url = 'https://poweroutage.us/area/state/texas'
response = requests.get(url=url)
response

<Response [200]>

In [210]:
# Parse the response content into HTML
soup = bs4.BeautifulSoup(response.content)
soup

<!DOCTYPE html>
<html lang="en">
<head>
<!--Client Side Site Config-->
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>Texas Power Outages Map</title> <link href="/content/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://poweroutage.us/area/state/texas" rel="canonical"/>
<!-- Windows Tile Config -->
<meta content="PowerOutage.us" name="application-name"/>
<meta content="#111" name="msapplication-TileColor"/>
<meta content="windows-icon-50x50.png" name="msapplication-square70x70logo"/>
<meta content="windows-icon-150x150.png" name="msapplication-square150x150logo"/>
<meta content="windows-icon-310x150.png" name="msapplication-wide310x150logo"/>
<meta content="windows-icon-310x310.png" name="msapplication-square310x310logo"/>
<meta content="frequency=10; polling-uri=content/windowslivetile1.xml; polling-uri2=content/windowslivetile2.xml;" name="msapplication-notification"/>
<!-- Apple Config -->
<meta

In [211]:
# bs4 has a find_all feature that looks for all instances of the specified HTML tag. 
# There are two tables in the HTML. We are interested in the first one, so we use bracket []
# indexing to grab it. 

table = soup.find_all('table')[0]
table

<table class="table-striped" width="100%">
<tr>
<th class="row">
<div class="hidden-xs hidden-sm col-sm-4" style="padding-left: 0px;">Provider</div>
<div class="text-right col-xs-6 col-md-3 nowrap" style="padding-left: 0px; padding-right: 0px;">Customers Tracked</div>
<div class="text-right col-xs-6 col-md-2 nowrap" style="padding-left: 0px; padding-right: 0px;">Customers Out</div>
<div class="text-right hidden-xs hidden-sm text-right col-xs-6 col-sm-3" style="padding-right: 0px;">Last Updated</div>
</th>
</tr>
<tr class=""><td class="row"><div class="col-xs-12 col-md-5" style="padding-left:0px;"><a href="/area/utility/380">American Electric Power Texas</a></div><div class="text-right col-xs-6 col-md-2" style="padding-left: 0px; padding-right: 0px;">1,052,836</div><div class="text-right col-xs-6 col-md-2" style="padding-left: 0px; padding-right: 0px;">398,106</div><div class="hidden-xs hidden-sm col-md-3 text-right datetime" nowrap="" style="padding-right:0px;"><small>2/16/2021 5:27:58

In [212]:
# Pandas has a built-in html parser that sometimes works to automatically get
# tables Unfortunately not for us. 
# We will have to parse out the information manually.

df = pd.read_html(str(table), header=None)
df

[   Provider Customers Tracked Customers Out Last Updated
 0   American Electric Power Texas1,052,836398,1062...   
 1   Austin Energy512,484209,2972/16/2021 5:29:58 P...   
 2   Bandera Electric Cooperative37,5396,5492/16/20...   
 3   Bluebonnet Electric Cooperative109,62536,1472/...   
 4   Bowie-Cass Electric Cooperative23,0333512/16/2...   
 ..                                                ...   
 57  Upshur Rural Electric Cooperative Corporation5...   
 58  Wharton County Electric Cooperative6,3014,6332...   
 59  Wise Electric Cooperative24,3894,5712/16/2021 ...   
 60  Wood County Electric Cooperative32,007892/16/2...   
 61   Xcel Energy253,17031,9282/16/2021 5:21:30 PM GMT   
 
 [62 rows x 1 columns]]

In [213]:
# Our table data lives in the td tag. Lets grab those. 
data = table.find_all('td')
data

[<td class="row"><div class="col-xs-12 col-md-5" style="padding-left:0px;"><a href="/area/utility/380">American Electric Power Texas</a></div><div class="text-right col-xs-6 col-md-2" style="padding-left: 0px; padding-right: 0px;">1,052,836</div><div class="text-right col-xs-6 col-md-2" style="padding-left: 0px; padding-right: 0px;">398,106</div><div class="hidden-xs hidden-sm col-md-3 text-right datetime" nowrap="" style="padding-right:0px;"><small>2/16/2021 5:27:58 PM GMT</small></div></td>,
 <td class="row"><div class="col-xs-12 col-md-5" style="padding-left:0px;"><a href="/area/utility/409">Austin Energy</a></div><div class="text-right col-xs-6 col-md-2" style="padding-left: 0px; padding-right: 0px;">512,484</div><div class="text-right col-xs-6 col-md-2" style="padding-left: 0px; padding-right: 0px;">209,297</div><div class="hidden-xs hidden-sm col-md-3 text-right datetime" nowrap="" style="padding-right:0px;"><small>2/16/2021 5:29:58 PM GMT</small></div></td>,
 <td class="row"><di

In [89]:
# We can use nested for loops to create dictionaries that pull out the information. 

columns = ['Provider', 'Customers Tracked', 'Customers Out', 'Last Updated']

records = []
for row in data:
    record = {}
    for i, div in enumerate(row):
        record.update({columns[i]: div.text})
    records.append(record)
records

[{'Provider': 'American Electric Power Texas',
  'Customers Tracked': '1,052,836',
  'Customers Out': '399,731',
  'Last Updated': '2/16/2021 4:16:48 PM GMT'},
 {'Provider': 'Austin Energy',
  'Customers Tracked': '512,484',
  'Customers Out': '219,928',
  'Last Updated': '2/16/2021 4:17:38 PM GMT'},
 {'Provider': 'Bandera Electric Cooperative',
  'Customers Tracked': '24,160',
  'Customers Out': '3,680',
  'Last Updated': '2/16/2021 4:18:18 PM GMT'},
 {'Provider': 'Bluebonnet Electric Cooperative',
  'Customers Tracked': '109,625',
  'Customers Out': '22,099',
  'Last Updated': '2/16/2021 4:18:27 PM GMT'},
 {'Provider': 'Bowie-Cass Electric Cooperative',
  'Customers Tracked': '23,033',
  'Customers Out': '156',
  'Last Updated': '2/16/2021 4:18:28 PM GMT'},
 {'Provider': 'Bryan Texas Utilities',
  'Customers Tracked': '62,397',
  'Customers Out': '24,218',
  'Last Updated': '2/16/2021 4:18:31 PM GMT'},
 {'Provider': 'CenterPoint Energy',
  'Customers Tracked': '2,563,940',
  'Custome

In [207]:
# Now we can put it into a dataframe. I'm using lambda functions to turn strings into ints
# I'm using pd.to_datetime to ensure the Last Updated column is in a format that pandas likes.

df = pd.DataFrame(records)
df['Customers Tracked'] = df['Customers Tracked'].apply(lambda x: int(x.replace(',', '')))
df['Customers Out'] = df['Customers Out'].apply(lambda x: int(x.replace(',', '')))
df['Last Updated'] = pd.to_datetime(df['Last Updated'].astype('datetime64[ns]'))
df

,Provider,Customers Tracked,Customers Out,Last Updated
0,American Electric Power Texas,1052836,399731,2021-02-16 16:16:48
1,Austin Energy,512484,219928,2021-02-16 16:17:38
2,Bandera Electric Cooperative,24160,3680,2021-02-16 16:18:18
3,Bluebonnet Electric Cooperative,109625,22099,2021-02-16 16:18:27
4,Bowie-Cass Electric Cooperative,23033,156,2021-02-16 16:18:28
...,...,...,...,...
57,Upshur Rural Electric Cooperative Corporation,57667,6785,2021-02-16 16:18:22
58,Wharton County Electric Cooperative,6301,4626,2021-02-16 16:18:22
59,Wise Electric Cooperative,24389,4162,2021-02-16 16:18:13
60,Wood County Electric Cooperative,32007,270,2021-02-16 16:18:40
